In [1]:
import numpy as np
from scipy.optimize import fsolve

# Given constants
Q_L = 1000 / 60 / 1000  # m³/s (from 1000 L/min)
mu = 1e-3               # Pa·s
rho = 998               # kg/m³
g = 9.81                # m/s²
D_O2_water = 2e-9       # m²/s
L = 0.6                 # m
m = 0.0318              # Henry's constant (dimensionless)
CO2_G = 8.59            # mol/m³ (O2 in gas)
CO2_in_mgL = 5          # mg/L
CO2_out_mgL = 8         # mg/L

# Convert mg/L to mol/m³ (O2 molar mass = 32 g/mol = 32,000 mg/mol)
CO2_in = CO2_in_mgL / 32000 * 1e3  # mol/m³
CO2_out = CO2_out_mgL / 32000 * 1e3  # mol/m³

# Compute NO2 (molar flow rate of O2)
NO2 = Q_L * (CO2_out - CO2_in)  # mol/s

# Terminal velocity helper
def vz_max(delta):
    return rho * g * delta**2 / (2 * mu)

# Main function to solve: returns the error in NO2
def equation_to_solve(A):
    A = float(A)
    delta = (3 * mu * Q_L / (g * rho * A))**(1/3)
    vz = vz_max(delta)
    CO2_Li = NO2 * np.sqrt(np.pi * L) / (A * np.sqrt(2 * D_O2_water * vz))
    ln_term = np.log((CO2_Li - CO2_in) / (CO2_Li - CO2_out))
    kg = m * Q_L / A * ln_term
    NO2_calc = A * kg * (CO2_G - m * CO2_Li)
    return NO2_calc - NO2

# Initial guess for A
A_guess = 0.1

# Solve for A
A_solution = fsolve(equation_to_solve, A_guess)[0]

# Calculate required number of tubes if each has D = 0.03 m and L = 0.6 m
D_tube = 0.03
A_tube = np.pi * D_tube * L
n_tubes = np.ceil(A_solution / A_tube)

# Output results
print(f"Required transfer area A: {A_solution:.4f} m²")
print(f"Area per tube (D = 3 cm, L = 0.6 m): {A_tube:.4f} m²")
print(f"Number of tubes needed: {int(n_tubes)}")


Required transfer area A: 80.2505 m²
Area per tube (D = 3 cm, L = 0.6 m): 0.0565 m²
Number of tubes needed: 1420


/var/folders/h_/r3lv2bc576qbm4jbyp94hfxr0000gn/T/ipykernel_87674/850248306.py:29: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  A = float(A)
